In [1]:
import dotenv
import os

dotenv.load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_community.document_loaders import PyPDFLoader

book_path = '../data/raw/prueba-frankenstein/pauta_prueba_2.pdf'

loader:PyPDFLoader = PyPDFLoader(book_path)
pages = loader.load_and_split()

In [3]:
complete_text = ' '.join([page.page_content.replace('\n', ' ') for page in pages])
len(complete_text.split(' '))

1437

In [4]:
from langchain.prompts import ChatPromptTemplate

prompt = """Eres un profesor experto en todas las materias del colegio.
Extrae las preguntas y respuestas que encuentres en el siguiente texto:
texto = {texto}
Separa cada pregunta y su respuesta con ;;.
Separa cada par pregunta-respuesta con un salto de línea.
En caso de que la pregunta venga con contexto, incluyelo con la pregunta.
Ejemplo: Pregunta1;;Respuesta1||Pregunta2;;Respuesta2
"""
prompt = ChatPromptTemplate.from_template(prompt)

In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

In [6]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

llm_chain = (
    {"texto": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [7]:
output = llm_chain.invoke({"texto": complete_text})
output

'1 Considera las circunstancias en las que el padre de Frankenstein conoce a su futura esposa y se convierte de hecho en el padre de Elizabeth Lavenza. ¿Qué indican sobre su forma de ser?;;Que se trata de un espíritu generoso y caritativo, que acude en ayuda de sus seres queridos cuando estos lo necesitan.\n\n2 Desde niño Victor Frankenstein manifiesta una gran pasión por el conocimiento y la investigación. Busca al final de este capítulo una oración que resuma esta actitud.;;“El mundo era para mí un secreto que deseaba descubrir”.\n\n1 ¿Por qué llama el protagonista “disparatadas fantasías” a las teorías de Agrippa, Paracelso y Alberto Magno?;;Porque se trata de teorías sobradamente superadas al comienzo del siglo XIX, que están más cerca de la magia y de la brujería que de la ciencia moderna.\n\n2 En este capítulo queda completado el entorno familiar del protagonista. ¿Quiénes lo componen? ¿Cómo es su infancia en el seno de este entorno?;;Además de sus cariñosos padres, Victor tiene 

In [11]:
question_and_answers = []
for question_answer in output.split('\n\n'):
    question, answer = question_answer.split(';;')
    question_and_answers.append({'question': question, 'answer': answer})

question_and_answers

[{'question': '1 Considera las circunstancias en las que el padre de Frankenstein conoce a su futura esposa y se convierte de hecho en el padre de Elizabeth Lavenza. ¿Qué indican sobre su forma de ser?',
  'answer': 'Que se trata de un espíritu generoso y caritativo, que acude en ayuda de sus seres queridos cuando estos lo necesitan.'},
 {'question': '2 Desde niño Victor Frankenstein manifiesta una gran pasión por el conocimiento y la investigación. Busca al final de este capítulo una oración que resuma esta actitud.',
  'answer': '“El mundo era para mí un secreto que deseaba descubrir”.'},
 {'question': '1 ¿Por qué llama el protagonista “disparatadas fantasías” a las teorías de Agrippa, Paracelso y Alberto Magno?',
  'answer': 'Porque se trata de teorías sobradamente superadas al comienzo del siglo XIX, que están más cerca de la magia y de la brujería que de la ciencia moderna.'},
 {'question': '2 En este capítulo queda completado el entorno familiar del protagonista. ¿Quiénes lo comp